In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable

open_api_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client

openai_client = OpenAI(api_key = open_api_key)



In [3]:
# 15

from modules.pdf_extraction import select_pdf_file
from modules.pdf_extraction import extract_text_from_pages

pdf_path = select_pdf_file()
page_no = 0
extracted_text = extract_text_from_pages(pdf_path, pages=page_no)


2025-02-26 19:43:44,456 - INFO - Opening file selection dialog.
2025-02-26 19:43:50,400 - INFO - Selected PDF file: C:/Users/derri/Downloads/page0025.pdf


In [4]:
import pymupdf 
## 111
import base64
import requests

import importlib
import modules.llm
import modules.pdf_extraction


from modules.pdf_extraction import get_page_pixel_data



# this may have to happen on a page by page basis

base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                    dpi = 500, image_type = 'png')

In [5]:
# import pymupdf

# # Open some document, for example a PDF (could also be EPUB, XPS, etc.)
# doc = pymupdf.open(pdf_path)

# # Load a desired page. This works via 0-based numbers
# page = doc[0]  # this is the first page

# # Look for tables on this page and display the table count
# tabs = page.find_tables()
# num_tables = len(tabs.tables)

In [7]:
import modules.llm
importlib.reload(modules.llm)
from modules.llm import table_identification_llm

output_table_identification_llm = await table_identification_llm(text_input=extracted_text,  base64_image=base64_image, open_api_key=open_api_key)


In [9]:
output_table_identification_llm

'1. Number of Tables on the Page: 5  \n\n2. Table Headers: "JIC 37° Male Inserts - Top left of the page" || "JIC 37° Swivel Female Inserts - Top center of the page" || "JIC 37° Female Swept Elbow Inserts - Top right of the page" || "JIC 37° Female Swept Elbow Inserts - Bottom left of the page" || "JIC 37° Female Compact 90° Elbow Inserts - Bottom right of the page"\n\n3. Table Location for each table: [Table is present in both the image and the text document] || [Table is present in both the image and the text document] || [Table is present in both the image and the text document] || [Table is present in both the image and the text document] || [Table is present in both the image and the text document]'

In [10]:
importlib.reload(modules.pdf_extraction)
from modules.pdf_extraction import get_validated_table_info
num_tables, table_headers, table_location, confidence_score_0 = await get_validated_table_info(text_input=extracted_text, open_api_key=open_api_key, base64_image= base64_image)

5 5
Initial headers match or same number of tables


In [ ]:
# from modules.pdf_extraction import table_identification_llm


# output = await table_identification_llm(text_input=extracted_text, base64_image=base64_image, open_api_key=open_api_key)


In [12]:
if num_tables == 0:
    raise ValueError("No tables found on the page")



In [9]:
# add in control for no tables found
# table_with_index = " || ".join([f"index {i}: {header}" for i, header in enumerate(table_headers)])

In [10]:
# tables_data = [f"[{i}]" for i in table_headers]

In [13]:
extracted_text

'\n\n--- Page 1 ---\n\n19\n1.01  Hydraulic Hose & Inserts\nHydraulics\n1\nJIC\n Hexavalent chromium free plating\nJIC 37¡ Male Inserts\n0309\nThread\nTo Suit Hose ID\nJM0403CF\n7/16”\n3/16”\nJM0404CF\n7/16”\n1/4”\nJM0504CF\n1/2”\n1/4”\nJM0505CF\n1/2”\n5/16”\nJM0604CF\n9/16”\n1/4”\nJM0605CF\n9/16”\n5/16”\nJM0606CF\n9/16”\n3/8”\nJM0608CF\n9/16”\n1/2”\nJM0805CF\n3/4”\n5/16”\nJM0806CF\n3/4”\n3/8”\nJM0808CF\n3/4”\n1/2”\nJM1006CF\n7/8”\n3/8”\nJM1008CF\n7/8”\n1/2”\nJM1010CF\n7/8”\n5/8”\nJM1012CF\n7/8”\n3/4”\nJM1208CF\n3/4”\n1/2”\nJM1210CF\n1.1/16”\n5/8”\nJM1212CF\n1.1/16”\n3/4”\nJM1216CF\n1.1/16”\n1”\nJM1412CF\n1.3/16”\n3/4”\nJM1416CF\n1.3/16”\n1”\nJM1612CF\n1.5/16”\n3/4”\nJM1616CF\n1.5/16”\n1”\nJM2016CF\n1.5/8”\n1”\nJM2020CF\n1.5/8”\n1.1/4”\nJM2420CF\n1.7/8”\n1.1/4”\nJM2424CF\n1.7/8”\n1.1/2”\nJM3232CF\n2.1/2”\n2”\nJIC 37¡ Swivel Female Inserts\n0309\nThread\nTo Suit Hose ID\nJF0403CF\n7/16”\n3/16”\nJF0404CF\n7/16”\n1/4”\nJF0406CF\n7/16”\n3/8\nJF0503CF\n1/2”\n3/16”\nJF0504CF\n1/2”\n1/4”\nJF05

In [16]:
import json
import pandas as pd
import re
import importlib
import modules.pdf_extraction
importlib.reload(modules.pdf_extraction)
from modules.pdf_extraction import process_tables_to_df


user_input = "I would like to get all the data text from the table"

output_1 = await process_tables_to_df(
                     user_text=user_input,
                     table_location = table_location,
                     table_headers = table_headers,
                     extracted_text=extracted_text, 
                     base64_image=base64_image, 
                     open_api_key=open_api_key)


output_1

,0309,Thread,To Suit Hose Id,table_header_descriptor
0,JM0403CF,7/16”,3/16”,"""JIC 37° Male Inserts - Can be found on the to..."
1,JM0404CF,7/16”,1/4”,"""JIC 37° Male Inserts - Can be found on the to..."
2,JM0504CF,1/2”,1/4”,"""JIC 37° Male Inserts - Can be found on the to..."
3,JM0505CF,1/2”,5/16”,"""JIC 37° Male Inserts - Can be found on the to..."
4,JM0604CF,9/16”,1/4”,"""JIC 37° Male Inserts - Can be found on the to..."
...,...,...,...,...
120,JF0890K08CF,3/4”,1/2”,"""JIC 37° Female Compact 90° Elbow Inserts - Ca..."
121,JF1090K08CF,7/8”,1/2”,"""JIC 37° Female Compact 90° Elbow Inserts - Ca..."
122,JF1090K10CF,7/8”,5/8”,"""JIC 37° Female Compact 90° Elbow Inserts - Ca..."
123,JF1290K12CF,1.1/16”,3/4”,"""JIC 37° Female Compact 90° Elbow Inserts - Ca..."


In [12]:
# # add in try and except for error handling

# import json
# import pandas as pd
# import re
# import importlib
# import modules.llm
# importlib.reload(modules.llm)
# from modules.llm import vision_llm_parser

# user_input = "I would like to get all the data text from the table"

# output = await vision_llm_parser(
#                 user_text=user_input,
#                 text_input=extracted_text,
#                 table_to_target=table_headers[1],
#                 base64_image=base64_image,
#                 open_api_key=open_api_key,
#                 model='gpt-4o'
#             )

# # from modules.pdf_extraction import parse_variable_data_to_df
# # df_0 = parse_variable_data_to_df(output)
# # df_0

# # def extract_list_from_string(text):
# #     # Find the content between the first [ and last ], including the brackets
# #     match = re.search(r'(\[.*\])', text, re.DOTALL)
# #     if match:
# #         # Return the full string including brackets
# #         return match.group(1)
# #     return None

# # extracted_dict = extract_list_from_string(output)


# # data = json.loads(extracted_dict)
# # df = pd.DataFrame(data)
# # df

In [8]:
# pdf_path = select_pdf_file()

2025-02-24 20:48:19,294 - INFO - Opening file selection dialog.
2025-02-24 20:48:21,790 - INFO - Selected PDF file: C:/Users/derri/Downloads/page0025.pdf


In [9]:
# import pymupdf

# # Open some document, for example a PDF (could also be EPUB, XPS, etc.)
# doc = pymupdf.open(pdf_path)

# # Load a desired page. This works via 0-based numbers
# page = doc[0]  # this is the first page

# # Look for tables on this page and display the table count
# tabs = page.find_tables()
# print(f"{len(tabs.tables)} table(s) on {page}")

# # We will see a message like "1 table(s) on page 0 of input.pdf"

6 table(s) on page 0 of C:/Users/derri/Downloads/page0025.pdf


In [ ]:
################

In [1]:
from modules.pdf_extraction import select_pdf_file

from modules.pdf_extraction import get_validated_table_info
from modules.pdf_extraction import get_page_pixel_data
from modules.pdf_extraction import process_tables_to_df
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import asyncio
import os
import pymupdf
import time



# to do.. table names headers can be dupilcated may be helpful to use say table 1,2, etc to deitigusih tem. 
# use pdf plumber  page.find_tables() and gpt table count for confidence calculation. 
# the same table can be extraected twice or more if the header is not clear
# extract header witout the text 

file_name = 'test_7'    
user_text='Extract all data from the table(s) the header'

# 1. Load Credientials

# Load environment variables from the .env file
load_dotenv()
# Get the API key from the environment variable
open_api_key = os.getenv('OPENAI_API_KEY')
# Initialize OpenAI client
openai_client = OpenAI(api_key = open_api_key)





# 2. Select PDF file and extract text
pdf_path = select_pdf_file()
doc = pymupdf.open(pdf_path)
total_pages = doc.page_count  # total number of pages in the document
page_indices = range(total_pages)

# page_indices can be a list of page numbers to process


# Start timing
start_time = time.time()

async def process_page():
    tasks = []
    results_output = []
    # Create all tasks first 
    async with asyncio.TaskGroup() as tg:
        for page_no in page_indices:
            page = doc.load_page(page_no)
            extracted_text = page.get_text()
            
            # extracted_text = extract_text_from_pages(pdf_path, pages=page_no)
            base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                                dpi = 500, image_type = 'png')
        
            num_tables, table_headers, table_location, confidence_score_0 = await get_validated_table_info(
                text_input=extracted_text, 
                open_api_key=open_api_key, 
                base64_image=base64_image
            )

            if num_tables == 0:
                print(f"No tables found on page {page_no + 1}, skipping...")
                continue
    
            tasks.append(tg.create_task(process_tables_to_df(
                table_headers, 
                table_location,
                user_text, 
                extracted_text, 
                base64_image, 
                open_api_key,
                page_number=page_no)))
            
        # Await all tasks to complete
        for task in tasks:
            results_output.append(await task)
    
    if not results_output:
        raise ValueError("No tables found on any of the processed pages")
            
    # df_out_1 = pd.concat(results_output, ignore_index=True)
    return results_output


output_final = await process_page()


2025-02-27 21:45:01,138 - INFO - Opening file selection dialog.
2025-02-27 21:45:03,431 - INFO - Selected PDF file: C:/Users/derri/Downloads/3 in one.pdf


3 3
Initial headers match or same number of tables
8 6
No matches found. Returning third run results for table_headers
7 8
No matches found. Returning third run results for table_headers


In [2]:
import pandas as pd
import itertools


def write_output_final(output_final, excel_path, option=1, gap_rows=2):
    """
    Writes nested lists of DataFrames (`output_final`) to Excel in 3 different ways.

    :param output_final: A list of lists of DataFrames. 
                        e.g. [
                            [df0_0, df0_1, df0_2],  # first "page"
                            [df1_0, df1_1],        # second "page"
                            ...
                        ]
    :param excel_path: Output Excel filename/path
    :param option: Choose 1 of 3 write modes:
                   1 = Horizontally merge (side-by-side) all DataFrames into one wide table (one sheet)
                   2 = Each top-level group on its own sheet, with `gap_rows` blank rows between sub-DataFrames
                   3 = Flatten all DataFrames onto one sheet vertically, with `gap_rows` blank rows between them
    :param gap_rows: How many blank rows to insert between tables (used in options 2 and 3).
    """
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        
        if option == 1:
            # ---------------------------------------------------------
            # (1) Horizontally merge *all* DataFrames into ONE wide table
            # ---------------------------------------------------------
            # Flatten the nested list into a single list of DataFrames
            all_dfs = list(itertools.chain.from_iterable(output_final))
            
            # Concatenate them horizontally (side-by-side)
            # By default, rows are aligned on the index; 
            # if DataFrames have different row indices, you may need join='outer' or join='inner'
            merged_df = pd.concat(all_dfs, axis=0)
            merged_df.to_excel(writer, sheet_name="AllTablesMerged", index=False)
            
        elif option == 2:
            # ---------------------------------------------------------
            # (2) Each top-level group on a DIFFERENT sheet,
            #     with gap_rows between each sub-DataFrame
            # ---------------------------------------------------------
            for page_idx, df_group in enumerate(output_final):
                # Each top-level group is a "page" => one sheet
                sheet_name = f"Page_{page_idx+1}"
                start_row = 0
                
                for df in df_group:
                    df.to_excel(writer, sheet_name=sheet_name, startrow=start_row, index=False)
                    # Shift down for the next DF: table rows + header row + gap_rows
                    start_row += len(df) + 1 + gap_rows
                    
        elif option == 3:
            # ---------------------------------------------------------
            # (3) Flatten all DataFrames on ONE sheet (stacked vertically),
            #     with gap_rows rows between each
            # ---------------------------------------------------------
            
            all_dfs = list(itertools.chain.from_iterable(output_final))
            
            sheet_name = "AllTablesWithGaps"
            start_row = 0
            for df in all_dfs:
                df.to_excel(writer, sheet_name=sheet_name, startrow=start_row, index=False)
                start_row += len(df) + 1 + gap_rows
                
        else:
            raise ValueError("Invalid `option` - must be 1, 2, or 3.")




In [3]:
write_output_final(output_final, excel_path='files/AllTables11.xlsx', option=1, gap_rows=2)

In [53]:

# import itertools
# all_dfs = list(itertools.chain.from_iterable(output_final))
# combined_dfs = pd.concat(all_dfs)


# combined_dfs['descriptor_label'] = (
#     combined_dfs.groupby('page_number')['table_header_descriptor']
#     .transform(lambda x: pd.factorize(x)[0] + 1)
# )


# combined_dfs['page_table'] = (
#     'page ' + combined_dfs['page_number'].astype(str) +
#     ' table ' + combined_dfs['descriptor_label'].astype(str)
# )


In [ ]:
# mapping_dict = combined_dfs.groupby('table_header_descriptor')['page_table'].apply(list).to_dict()
# mapping_dict


In [62]:
# df['table_header_descriptor'].map(mapping_dict)

In [36]:
# import itertools


# all_dfs = list(itertools.chain.from_iterable(output_final))
# combined_dfs = pd.concat(all_dfs)
# combined_dfs['table_header_descriptor_factorized'], _ = pd.factorize(combined_dfs['table_header_descriptor'])



# unique_headers = combined_dfs['table_header_descriptor_factorized'].unique()
# header_to_num = {
#     header: f"Page {combined_dfs['page_number'] + 1} Table {i+1}"
#     for i, header in enumerate(unique_headers)
# }

In [10]:
# # Generate a "page_table" label using unique descriptors
# unique_headers = df['table_header_descriptor'].unique()
# header_to_num = {
#     header: f"Page {df['page_number'] + 1} Table {i+1}"
#     for i, header in enumerate(unique_headers)
# }
# df['page_table'] = df['table_header_descriptor'].map(header_to_num)

# # df.drop(columns=['table_header_descriptor'], inplace=True)


In [11]:
# write combined tables to excel

df_combined = pd.concat(output_final[0], ignore_index=True)
df_combined.to_excel(f'files/{file_name}.xlsx', index=False)


In [ ]:
from modules.pdf_extraction import write_split_tables_to_excel

# Call the function with output_final[0]
write_split_tables_to_excel(output_final[0],output_path='files/test_output.xlsx')
